# Imports

In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.2.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install keras

In [ ]:
!pip install dill

In [ ]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall scikit-learn


In [ ]:
!pip uninstall -y scipy scikit-learn numpy
!pip install numpy==1.24.4 scipy==1.10.1 scikit-learn==1.3.2


In [ ]:
# Import required libraries for building a neural network, data manipulation, and evaluation metrics
from keras.models import Sequential               # Sequential model to stack layers linearly
from keras.layers import Dense                    # Dense layer to create fully connected layers
from keras.optimizers import Adadelta             # Optimizer used to minimize the loss function
import numpy as np                                # NumPy for numerical operations
import pandas as pd                               # Pandas for data manipulation
from sklearn.preprocessing import MinMaxScaler    # Scaler to normalize features to a specific range
import matplotlib.pyplot as plt                   # Matplotlib for plotting
from sklearn.metrics import (                     # Metrics for model evaluation
    f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
)
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split  # For splitting datasets
from sklearn.metrics import confusion_matrix, accuracy_score
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from keras.models import load_model
import os
from sklearn.utils import resample
from sklearn.cluster import DBSCAN
import pickle
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from matplotlib.dates import DateFormatter

In [ ]:
#NN-AE model for data without augmentation
# Define the path to the model file
model_path = '/content/drive/MyDrive/Notebook_with_plots/autoencoder_model(original_batch_size_128_50epchs3).keras'

# Check if the file exists
if os.path.exists(model_path):
    # Load the model
    model = load_model(model_path, compile=False)
else:
    print("Model file not found.")



    #Load model for IF
# Define the path to the model file
best_isolation_forest_model_path = '/content/drive/MyDrive/Notebook_with_plots/best_isolation_forest_model_original.pkl'

# Check if the file exists
if os.path.exists(best_isolation_forest_model_path):
    # Load the saved model using dill
    import dill
    with open(best_isolation_forest_model_path, "rb") as model_file:
        loaded_isolation_forest_IF_original = dill.load(model_file)
else:
    print("Model file not found.")



#Load model for DBSCAN
# Define the paths to the files
best_params_path = '/content/drive/MyDrive/Notebook_with_plots/best_dbscan_params_90s.pkl'
dbscan_results_path = '/content/drive/MyDrive/Notebook_with_plots/dbscan_results_90s.pkl'

if os.path.exists(best_params_path) and os.path.exists(dbscan_results_path):
    # Load the best parameters and evaluation results using dill
    import dill
    with open(best_params_path, "rb") as params_file:
        best_params = dill.load(params_file)
    with open(dbscan_results_path, "rb") as results_file:
        dbscan_results_DBSCAN_Original = dill.load(results_file)
else:
    print("One or more files not found.")


#Load model for LOF
# Define the path to the file
best_lof_params_path = '/content/drive/MyDrive/Notebook_with_plots/best_lof_params_original.pkl'
import os
if os.path.exists(best_lof_params_path):
    with open(best_lof_params_path, "rb") as params_file:
        loaded_best_params_LOF_Original = dill.load(params_file)
else:
    print("File not found.")



#Load data without augmentation
# Define the path to the data file
data_path = '/content/drive/MyDrive/Notebook_with_plots/data_without_augmentation.txt'

# Check if the file exists
import os
if os.path.exists(data_path):
    # Load the data
    import pandas as pd
    data = pd.read_csv(data_path, sep='\t')
else:
    print("Data file not found.")




# Define the path to the data files
test_data_path = '/content/drive/MyDrive/Notebook_with_plots/test_data_All.txt'
train_data_path = '/content/drive/MyDrive/Notebook_with_plots/training_NN-AE(original).txt'
validation_data_path = '/content/drive/MyDrive/Notebook_with_plots/validation_NN-AE(original).txt'

# Check if the files exist
import os
if os.path.exists(test_data_path) and os.path.exists(train_data_path) and os.path.exists(validation_data_path):
    # Load the data
    import pandas as pd
    test_data = pd.read_csv(test_data_path, sep='\t')
    train_data_final = pd.read_csv(train_data_path, sep='\t')
    validation_data = pd.read_csv(validation_data_path, sep='\t')
else:
    print("One or more data files not found.")

# Global Functions

In [ ]:
def calculate_metrics_for_multiple_percentiles(
    model,
    X_train_features_scaled, Y_train_labels,  # Training set
    X_test_features_scaled, Y_test_labels,    # Test set
    X_validation_set_2_features_scaled, Y_validation_set_2_labels,  # Validation set 2 for thresholds
    percentiles = [65]#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20,21, 22, 23, 24, 25, 26, 27,
                   #28, 29, 30,31, 32, 33, 34, 35, 36, 37, 38, 39, 40,41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
                   #53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,71, 72, 73, 74, 75, 76, 77,
                   #78, 79, 80,81, 82, 83, 84, 85, 86, 87, 88, 89, 90,91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
):
    # Initialize lists to store validation thresholds and results
    validation_thresholds = []
    results = []

    # Step 1: Calculate validation thresholds for each percentile using validation set 2
    for percentile in percentiles:
        # Compute reconstruction errors for validation set 2
        val_predictions_2 = model.predict(X_validation_set_2_features_scaled)
        val_reconstruction_errors_2 = np.mean(np.square(X_validation_set_2_features_scaled - val_predictions_2), axis=1)

        # Calculate threshold for the current percentile
        threshold = np.percentile(val_reconstruction_errors_2, percentile)
        validation_thresholds.append(threshold)

    # Step 2: Validate using the calculated thresholds on the validation set
    for idx, percentile in enumerate(percentiles):
        threshold = validation_thresholds[idx]

        # Compute reconstruction errors for the validation set
        val_predictions = model.predict(X_validation_set_2_features_scaled)
        val_reconstruction_errors = np.mean(np.square(X_validation_set_2_features_scaled - val_predictions), axis=1)

        # Apply the threshold to classify validation samples
        y_val_pred = (val_reconstruction_errors > threshold).astype(int)

        # Calculate metrics for the validation set
        val_accuracy = accuracy_score(Y_validation_set_2_labels, y_val_pred)
        precision = precision_score(Y_validation_set_2_labels, y_val_pred)
        recall = recall_score(Y_validation_set_2_labels, y_val_pred)
        f1 = f1_score(Y_validation_set_2_labels, y_val_pred)

        # Specificity: True Negatives / (True Negatives + False Positives)
        tn, fp, fn, tp = confusion_matrix(Y_validation_set_2_labels, y_val_pred).ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        # Store results for this percentile
        results.append({
            'Percentile': percentile,
            'Optimal Threshold': threshold,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Specificity': specificity,
            'Accuracy': val_accuracy
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


In [ ]:
def plot_reconstruction_error_with_shuffle(
    model,
    X_test_features_scaled,
    Y_test_labels,
    optimal_thresholds,
    percentiles,
    save_folder="reconstruction_error_shuffled_plots"
):

    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Compute reconstruction errors for the test set
    test_predictions = model.predict(X_test_features_scaled)
    test_reconstruction_errors = np.mean(np.square(X_test_features_scaled - test_predictions), axis=1)

    # Shuffle the data indices
    shuffled_indices = np.random.permutation(len(Y_test_labels))
    shuffled_labels = Y_test_labels[shuffled_indices]
    shuffled_reconstruction_errors = test_reconstruction_errors[shuffled_indices]

    # Plot for each threshold
    for idx, (threshold, percentile) in enumerate(zip(optimal_thresholds, percentiles)):
        # Apply threshold to classify test samples
        y_test_pred = (shuffled_reconstruction_errors > threshold).astype(int)

        # Categorize points based on true/false positives/negatives
        true_anomalies = (shuffled_labels == 1) & (y_test_pred == 1)
        false_anomalies = (shuffled_labels == 0) & (y_test_pred == 1)
        true_normals = (shuffled_labels == 0) & (y_test_pred == 0)
        false_normals = (shuffled_labels == 1) & (y_test_pred == 0)

        # Plot reconstruction errors with different colors
        plt.figure(figsize=(12, 6))
        plt.scatter(np.arange(len(shuffled_labels))[true_anomalies], shuffled_reconstruction_errors[true_anomalies], color='purple', label='True Anomaly', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[false_anomalies], shuffled_reconstruction_errors[false_anomalies], color='red', label='False Anomaly', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[true_normals], shuffled_reconstruction_errors[true_normals], color='blue', label='True Normal', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[false_normals], shuffled_reconstruction_errors[false_normals], color='yellow', label='False Normal', s=10)

        # Add plot details
        plt.axhline(y=threshold, color='green', linestyle='--', label=f'Threshold: {threshold:.2f}')
        plt.title(f"Plot for reconstruction error")
        plt.xlabel("Data Index")
        plt.ylabel("Reconstruction Error")
        plt.legend()
        plt.tight_layout()

        # Save the figure as an image
        image_path = os.path.join(save_folder, f"reconstruction_error_percentile_{percentile}_shuffled.png")
        plt.savefig(image_path, bbox_inches='tight')

        # Display the figure
        plt.show()

In [ ]:
#Function to detect anomalies on new data using the model and threshold
def detect_anomalies_1(model_1, data_1, threshold_1):
    nn_input_data_1 =data_1
    predictions_1 = model_1.predict(nn_input_data_1)
    reconstruction_errors_1 = np.mean(np.square(nn_input_data_1 - predictions_1), axis=1)
    anomalies_1 = np.where(reconstruction_errors_1 > threshold_1, 1, 0)
    return anomalies_1

Temporal analysis of movements and detections Functions

In [ ]:
def plot_movement_with_anomalies_only(data, test_data, specific_fishid, start_date, end_date, model_name, anomaly_column, prediction_column):

    # Convert datetime columns to datetime type
    data['datetime'] = pd.to_datetime(data['datetime'])
    test_data['datetime'] = pd.to_datetime(test_data['datetime'])

    # Filter data within the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]
    filtered_test_data = test_data[(test_data['datetime'] >= start_date) & (test_data['datetime'] <= end_date)]

    # Filter specific fish ID
    all_movements = filtered_data[filtered_data['fishid'] == specific_fishid]
    filtered_test_data = filtered_test_data[filtered_test_data['fishid'] == specific_fishid]

    # Classify the results
    true_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 1)]
    false_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 0) & (filtered_test_data[prediction_column] == 1)]
    false_negatives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 0)]

    # Convert datetime to matplotlib format
    def to_mdates(series):
        return mdates.date2num(series)

    # Plotting
    plt.figure(figsize=(15, 6))

    # Plot all movements for context
    plt.plot_date(to_mdates(all_movements['datetime']), all_movements['lat'], 'b-', alpha=0.5, label='Fish Movement')

    # Plot false positives
    plt.plot_date(to_mdates(false_positives['datetime']), false_positives['lat'], 'ro', label='False Positives')

    # Plot false negatives
    plt.plot_date(to_mdates(false_negatives['datetime']), false_negatives['lat'], 'go', label='False Negatives')

    # Plot true positives
    plt.plot_date(to_mdates(true_positives['datetime']), true_positives['lat'], 'yo', label='True Positives')

    # Add title and labels
    plt.title(f'{model_name}: Fish Movement and Anomalies for FishID {specific_fishid}')
    plt.xlabel('Date')
    plt.ylabel('Latitude')

    # Show legend
    plt.legend()

    # Format x-axis
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gcf().autofmt_xdate()

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Adjust layout
    plt.tight_layout()

    # Display the plot
    plt.show()

Generating acoustic telemetry data from real data(Fish detection fuctions)

In [ ]:
def augment_data_with_sampling(normal_data, median_sampling_rate_seconds=8):
    # Ensure the data is sorted by datetime
    data1 = normal_data.copy()
    data1['datetime'] = pd.to_datetime(data1['datetime'])
    data1 = data1.sort_values('datetime').reset_index(drop=True)

    # Create a copy of the original data to modify it in place
    augmented_data = data1.copy()

    # Iterate through the dataset and add new instances where needed
    i = 0
    while i < len(augmented_data) - 1:
        current_time = augmented_data.loc[i, 'datetime']
        next_time = augmented_data.loc[i + 1, 'datetime']
        time_diff = next_time - current_time

        if time_diff > pd.Timedelta(seconds=median_sampling_rate_seconds):
            samples_needed = int(time_diff.total_seconds() // median_sampling_rate_seconds)
            sampled_row = augmented_data.sample(n=1, replace=True, random_state=42).iloc[0].copy()

            for j in range(1, samples_needed + 1):
                new_time = current_time + pd.to_timedelta(j * median_sampling_rate_seconds, unit='s')
                if new_time >= next_time:
                    break
                new_row = sampled_row.copy()
                new_row['datetime'] = new_time
                augmented_data = pd.concat([
                    augmented_data.iloc[:i + 1],
                    pd.DataFrame([new_row]),
                    augmented_data.iloc[i + 1:]
                ]).reset_index(drop=True)
                i += 1

                # Re-evaluate the next interval after insertion
                current_time = new_time
                next_time = augmented_data.loc[i + 1, 'datetime']
                time_diff = next_time - current_time

                if time_diff <= pd.Timedelta(seconds=median_sampling_rate_seconds):
                    break

        i += 1

    # Drop any potential duplicates
    augmented_data = augmented_data.drop_duplicates(subset=['datetime']).reset_index(drop=True)

    return augmented_data

# Loading data and models




In [ ]:
#Test data for shallow models
training_data1=pd.concat([train_data_final, validation_data])

In [ ]:
def prepare_datasets_from_precomputed(train_data_final, test_data, validation_data, anomaly_column='Anomaly'):
    # Separate features and target variables
    columns_to_drop = [anomaly_column, 'fishid','datetime', 'tag_date', 'station', 'date', 'avg_time_between_detections',
                   'time_since_last_detection', 'time_since_last_detection_days'] # Drop 'fishid' and 'datetime'
    X_train_features = train_data_final.drop(columns=columns_to_drop).values
    Y_train_labels = train_data_final[anomaly_column].values
    X_test_features = test_data.drop(columns=columns_to_drop).values
    Y_test_labels = test_data[anomaly_column].values

    # Separate anomalies and normal data in validation data
    anomalies = validation_data[validation_data[anomaly_column] == 1]
    normal = validation_data[validation_data[anomaly_column] == 0]

    # Split normal data in validation set into two subsets
    normal_subset_1, normal_subset_2 = train_test_split(normal, test_size=0.5)

    # Create two balanced validation sets
    validation_set_1 = normal_subset_1
    validation_set_2 = pd.concat([anomalies, normal_subset_2])

    # Separate features and labels for validation sets
    X_validation_set_1_features = validation_set_1.drop(columns=columns_to_drop).values
    Y_validation_set_1_labels = validation_set_1[anomaly_column].values
    X_validation_set_2_features = validation_set_2.drop(columns=columns_to_drop).values
    Y_validation_set_2_labels = validation_set_2[anomaly_column].values

    # Scale feature values
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_features_scaled = scaler.fit_transform(X_train_features)
    X_test_features_scaled = scaler.transform(X_test_features)
    X_validation_set_1_features_scaled = scaler.transform(X_validation_set_1_features)
    X_validation_set_2_features_scaled = scaler.transform(X_validation_set_2_features)

    # Return all datasets as variables, including the full test_data
    return {
        "X_train_features_scaled": X_train_features_scaled,
        "X_test_features_scaled": X_test_features_scaled,
        "X_validation_set_1_features_scaled": X_validation_set_1_features_scaled,
        "X_validation_set_2_features_scaled": X_validation_set_2_features_scaled,
        "Y_train_labels": Y_train_labels,
        "Y_test_labels": Y_test_labels,
        "Y_validation_set_1_labels": Y_validation_set_1_labels,
        "Y_validation_set_2_labels": Y_validation_set_2_labels,
        "X_test_features": X_test_features,
        "test_data": test_data
    }

variables = prepare_datasets_from_precomputed(train_data_final, test_data, validation_data)

# Access variables
X_train_features_scaled = variables["X_train_features_scaled"]
X_test_features_scaled = variables["X_test_features_scaled"]
X_validation_set_1_features_scaled = variables["X_validation_set_1_features_scaled"]
X_validation_set_2_features_scaled = variables["X_validation_set_2_features_scaled"]
Y_train_labels = variables["Y_train_labels"]
Y_test_labels = variables["Y_test_labels"]
Y_validation_set_1_labels = variables["Y_validation_set_1_labels"]
Y_validation_set_2_labels = variables["Y_validation_set_2_labels"]

# Building and Fitting the model

In [ ]:
# Looking for optimal threshold for each percentile
results_df = calculate_metrics_for_multiple_percentiles(
    model=model,
    X_train_features_scaled=X_train_features_scaled,
    Y_train_labels=Y_train_labels,
    X_test_features_scaled=X_test_features_scaled,
    Y_test_labels=Y_test_labels,
    X_validation_set_2_features_scaled=X_validation_set_2_features_scaled,
    Y_validation_set_2_labels=Y_validation_set_2_labels
)

# Display the results dataframe
print(results_df)


In [ ]:
# Create a figure and axis to plot the dataframe
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust the size if needed
ax.axis('tight')
ax.axis('off')

# Render the dataframe as a table on the plot
ax.table(cellText=results_df.values, colLabels=results_df.columns, cellLoc='center', loc='center')

# Save the plot as a PDF
with PdfPages('results_df_Big_new.pdf') as pdf:
    pdf.savefig(fig, bbox_inches='tight')
    plt.close(fig)

print("DataFrame has been saved as PDF.")


In [ ]:
#Looking for optimal threshold for each percentile
results_df = calculate_metrics_for_multiple_percentiles(
    model=model,
    X_train_features_scaled=X_train_features_scaled,
    Y_train_labels=Y_train_labels,
    X_test_features_scaled=X_test_features_scaled,
    Y_test_labels=Y_test_labels,
    X_validation_set_2_features_scaled=X_validation_set_2_features_scaled,
    Y_validation_set_2_labels=Y_validation_set_2_labels
)
print(results_df)

In [ ]:
# Extract the metrics you need
percentiles = results_df['Percentile'].values
precision = results_df['Precision'].values
recall = results_df['Recall'].values
specificity = results_df['Specificity'].values

# Display the extracted metrics
print("Percentiles:", percentiles)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)

In [ ]:
def plot_reconstruction_error_with_multiple_thresholds(
    model,
    X_validation_set_2_features_scaled,
    Y_validation_set_2_labels_scaled,
    optimal_thresholds,
    percentiles,
    #additional_thresholds,  # List of additional thresholds to display on each plot
    save_folder="reconstruction_error_shuffled_plots"
):

    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Compute reconstruction errors for the test set
    test_predictions = model.predict(X_validation_set_2_features_scaled)
    test_reconstruction_errors = np.mean(np.square(X_validation_set_2_features_scaled - test_predictions), axis=1)

    # Shuffle the data indices
    shuffled_indices = np.random.permutation(len(Y_validation_set_2_labels_scaled))
    shuffled_labels = Y_validation_set_2_labels_scaled[shuffled_indices]
    shuffled_reconstruction_errors = test_reconstruction_errors[shuffled_indices]

    # Plot for each threshold
    for idx, (threshold, percentile) in enumerate(zip(optimal_thresholds, percentiles)):
        # Apply the main threshold to classify test samples
        y_test_pred = (shuffled_reconstruction_errors > threshold).astype(int)

        # Categorize points based on true/false positives/negatives
        true_anomalies = (shuffled_labels == 1) & (y_test_pred == 1)
        false_anomalies = (shuffled_labels == 0) & (y_test_pred == 1)
        true_normals = (shuffled_labels == 0) & (y_test_pred == 0)
        false_normals = (shuffled_labels == 1) & (y_test_pred == 0)

        # Plot reconstruction errors with different colors
        plt.figure(figsize=(6, 5))
        plt.scatter(np.arange(len(shuffled_labels))[true_anomalies]/1000, shuffled_reconstruction_errors[true_anomalies], color='purple', label='True Anomaly', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[false_anomalies]/1000, shuffled_reconstruction_errors[false_anomalies], color='red', label='False Anomaly', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[true_normals]/1000, shuffled_reconstruction_errors[true_normals], color='blue', label='True Normal', s=10)
        plt.scatter(np.arange(len(shuffled_labels))[false_normals]/1000, shuffled_reconstruction_errors[false_normals], color='yellow', label='False Normal', s=10)

        # Add horizontal lines for each threshold
        plt.axhline(y=threshold, color='green', linestyle='--', label=f'Threshold: {threshold:.2f}')

        # Add plot details
        plt.xlabel(r"Data Index [$\times 10^3$]",fontsize=8)
        plt.ylabel("Reconstruction Error",fontsize=8)
        plt.legend(loc="best",fontsize="8")
        plt.tight_layout()

        # Save the figure as an image
        image_path = os.path.join(save_folder, f"reconstruction_error_percentile_{percentile}_shuffled.png")
        plt.savefig(image_path, bbox_inches='tight')

        # Display the figure
        plt.show()

In [ ]:




optimal_thresholds = results_df['Optimal Threshold'].tolist()
percentiles = results_df['Percentile'].tolist()

# Call the function
plot_reconstruction_error_with_multiple_thresholds(
    model,
    X_validation_set_2_features_scaled,
    Y_validation_set_2_labels,
    optimal_thresholds,
    percentiles
)

# NN_AE Plots

In [ ]:
best_threshold_original = 0.120692
# Detect anomalies in the test data
anomalies_test_data = detect_anomalies_1(model, X_test_features_scaled, best_threshold_original)

# Prepare processed test data with 'datetime' and 'fishid'
processed_test_data = pd.DataFrame(variables["X_test_features_scaled"])
processed_test_data['Anomaly'] = variables["Y_test_labels"]
processed_test_data['Anomaly_Predicted'] = anomalies_test_data

# Add the datetime column back to processed_test_data
processed_test_data['datetime'] = variables["test_data"]['datetime']
processed_test_data['lat'] = variables["test_data"]['lat']
processed_test_data['fishid'] = variables["test_data"]['fishid']

# Evaluation metrics
y_true = processed_test_data['Anomaly']
y_pred = processed_test_data['Anomaly_Predicted']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=1)
recall = recall_score(y_true, y_pred, zero_division=1)
f1 = f1_score(y_true, y_pred, zero_division=1)
roc_auc = roc_auc_score(y_true, y_pred) if len(np.unique(y_true)) > 1 else None

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

# Specificity calculation
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Print the evaluation metrics
print("Model Evaluation Metrics on Test Data:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Specificity:", specificity)
if roc_auc is not None:
    print("ROC AUC:", roc_auc)
else:
    print("ROC AUC is not applicable. Only one class present in y_true.")

In [ ]:
def plot_movement_with_anomalies(data, test_data, start_date, end_date, model_name, anomaly_column, prediction_column):

    # Convert datetime columns to datetime type
    data['datetime'] = pd.to_datetime(data['datetime'])
    test_data['datetime'] = pd.to_datetime(test_data['datetime'])

    # Filter data within the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]
    filtered_test_data = test_data[(test_data['datetime'] >= start_date) & (test_data['datetime'] <= end_date)]

    # Classify the results
    true_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 1)]
    true_negatives = filtered_test_data[(filtered_test_data[anomaly_column] == 0) & (filtered_test_data[prediction_column] == 0)]
    false_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 0) & (filtered_test_data[prediction_column] == 1)]
    false_negatives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 0)]

    anomalies = filtered_test_data[filtered_test_data[anomaly_column] == 1]

    # Helper function to plot
    def plot_helper(title, highlight_data, highlight_label, highlight_style, color):
        plt.figure(figsize=(12, 4))
        plt.plot(filtered_data['datetime'], filtered_data['lat'], 'b-', alpha=0.3, label='Fish Movement')
        plt.plot(anomalies['datetime'], anomalies['lat'], 'ro', label='Anomalies')
        plt.plot(highlight_data['datetime'], highlight_data['lat'], highlight_style, color=color, label=highlight_label)

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gcf().autofmt_xdate()
        plt.title(f'{title} ({model_name})', fontsize=16)
        plt.xlabel('Date')
        plt.ylabel('Latitude', fontsize=16)
        plt.xticks(fontsize=12, rotation=45)
        plt.yticks(fontsize=12)
        plt.legend(loc='upper right', fontsize=16)
        plt.show()

    # Plotting different cases
    plot_helper('Fish Movement and True Anomalies Highlighted', true_positives, 'True Anomalies', 'x', 'm')
    plot_helper('Fish Movement and True Normals Highlighted', true_negatives, 'True Normals', '^', 'g')
    plot_helper('Fish Movement and False Anomalies Highlighted', false_positives, 'False Anomalies', 'x', 'y')
    plot_helper('Fish Movement and False Normals Highlighted', false_negatives, 'False Normals', '^', 'c')

In [ ]:
plot_movement_with_anomalies(
    data=data,
    test_data=processed_test_data,
    start_date='2016-01-01',
    end_date='2021-12-31',
    model_name='NN-AE',
    anomaly_column='Anomaly',
    prediction_column='Anomaly_Predicted'
)

In [ ]:
def plot_movement_with_anomalies2020(data, test_data, start_date, end_date, model_name, anomaly_column, prediction_column):

    # Convert datetime columns to datetime type
    data['datetime'] = pd.to_datetime(data['datetime'])
    test_data['datetime'] = pd.to_datetime(test_data['datetime'])

    # Filter data within the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]
    filtered_test_data = test_data[(test_data['datetime'] >= start_date) & (test_data['datetime'] <= end_date)]

    # Classify the results
    true_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 1)]
    true_negatives = filtered_test_data[(filtered_test_data[anomaly_column] == 0) & (filtered_test_data[prediction_column] == 0)]
    false_positives = filtered_test_data[(filtered_test_data[anomaly_column] == 0) & (filtered_test_data[prediction_column] == 1)]
    false_negatives = filtered_test_data[(filtered_test_data[anomaly_column] == 1) & (filtered_test_data[prediction_column] == 0)]

    anomalies = filtered_test_data[filtered_test_data[anomaly_column] == 1]

    #function to plot
    def plot_helper(title, highlight_data, highlight_label, highlight_style, color):
        plt.figure(figsize=(12, 4))
        plt.plot(filtered_data['datetime'], filtered_data['lat'], 'b-', alpha=0.3, label='Fish Movement')
        plt.plot(anomalies['datetime'], anomalies['lat'], 'ro', label='Anomalies')
        plt.plot(highlight_data['datetime'], highlight_data['lat'], highlight_style, color=color, label=highlight_label)

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gcf().autofmt_xdate()
        #plt.title(f'{title} ({model_name}) -2020',fontsize=16)
        plt.ylabel('Latitude',fontsize=16)
        plt.xticks(fontsize=12, rotation=45)
        plt.yticks(fontsize=12)
        #plt.legend(loc='best', fontsize=16)
        plt.show()

    # Plotting different cases
    plot_helper('Fish Movement and True Anomalies Highlighted', true_positives, 'True Anomalies', 'x', 'm')
    plot_helper('Fish Movement and True Normals Highlighted', true_negatives, 'True Normals', '^', 'g')
    plot_helper('Fish Movement and False Anomalies Highlighted', false_positives, 'False Anomalies', 'x', 'y')
    plot_helper('Fish Movement and False Normals Highlighted', false_negatives, 'False Normals', '^', 'c')

In [ ]:
plot_movement_with_anomalies2020(
    data=data,
    test_data=processed_test_data,
    start_date='2020-01-01',
    end_date='2020-12-31',
    model_name='NN_AE',
    anomaly_column='Anomaly',
    prediction_column='Anomaly_Predicted'
)

In [ ]:
# Neural Network Autoencoder
plot_movement_with_anomalies_only(
    data=data,
    test_data=processed_test_data,
    start_date='2016-01-01',
    end_date='2021-12-31',
    specific_fishid= 'A69-9001-23859a',
    model_name='NN_AE',
    anomaly_column='Anomaly',
    prediction_column='Anomaly_Predicted'
)

# Isolation Forest Plots

In [ ]:
# Feature selection for Isolation Forest
selected_features_IF_original = ['lat', 'lon', 'duration_in_same_station', 'num_detections', 'distance', 'num_days_detected', 'num_unique_stations', 'consecutive_missing_stations']
test_data_IF_original = test_data[selected_features_IF_original + ['datetime']]

# Ground truth labels (actual anomalies) from the testing set
y_true_model_IF_original = test_data.loc[test_data_IF_original.index, 'Anomaly']

# Predict anomalies using the loaded model
anomalies_IF_original = loaded_isolation_forest_IF_original.predict(test_data_IF_original.drop('datetime', axis=1))
anomalies_IF_original = anomalies_IF_original == -1  # Convert -1 to True for anomalies

# Avoid SettingWithCopyWarning by creating a copy
test_data_IF_original = test_data.copy()
test_data_IF_original['Isolation_Forest_Anomaly'] = anomalies_IF_original.astype(int)

# Model's predictions on the testing data using Isolation Forest
y_pred_model_IF_original = test_data_IF_original['Isolation_Forest_Anomaly']

# Evaluate the performance of Isolation Forest
accuracy_IF_original = accuracy_score(y_true_model_IF_original, y_pred_model_IF_original)
precision_IF_original = precision_score(y_true_model_IF_original, y_pred_model_IF_original)
recall_IF_original = recall_score(y_true_model_IF_original, y_pred_model_IF_original)
f1_IF_original = f1_score(y_true_model_IF_original, y_pred_model_IF_original)

# Confusion matrix
conf_matrix_IF_original = confusion_matrix(y_true_model_IF_original, y_pred_model_IF_original)
tn, fp, fn, tp = conf_matrix_IF_original.ravel()

# Calculate specificity
specificity_IF_original = tn / (tn + fp)

# Calculate AUC
# Ensure y_prob_model_IF_original contains anomaly scores or probabilities
auc_IF_original = roc_auc_score(y_true_model_IF_original, y_pred_model_IF_original)

# Print performance metrics
print("Isolation Forest Performance with Best Parameters:")
print("Accuracy:", accuracy_IF_original)
print("Precision:", precision_IF_original)
print("Recall:", recall_IF_original)
print("F1-score:", f1_IF_original)
print("Specificity:", specificity_IF_original)
print("AUC:", auc_IF_original)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_IF_original)

In [ ]:
plot_movement_with_anomalies(
    data=data,
    test_data=test_data_IF_original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    model_name='IF',
    anomaly_column='Anomaly',
    prediction_column='Isolation_Forest_Anomaly'
)

In [ ]:
plot_movement_with_anomalies2020(
    data=data,
    test_data=test_data_IF_original,
    start_date='2020-01-01',
    end_date='2020-12-31',
    model_name='IF',
    anomaly_column='Anomaly',
    prediction_column='Isolation_Forest_Anomaly'
)


In [ ]:
# Isolation Forest
plot_movement_with_anomalies_only(
    data=data,
    test_data=test_data_IF_original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    specific_fishid= 'A69-9001-23859a',
    model_name='IF',
    anomaly_column='Anomaly',
    prediction_column='Isolation_Forest_Anomaly'
)

# DBSCAN Plots

In [ ]:
# Ground truth labels (actual anomalies) from the testing set
y_true_DBSCAN_Original = test_data['Anomaly']

# Feature selection for DBSCAN
selected_features_DBSCAN_Original = [
    'receiver', 'lat', 'lon', 'duration_in_same_station', 'num_detections',
    'distance', 'num_days_detected', 'num_unique_stations', 'consecutive_missing_stations'
]

X_test_DBSCAN_Original = test_data[selected_features_DBSCAN_Original]

# Use the best parameters to detect anomalies
dbscan_best = DBSCAN(eps=0.5, min_samples=10)
clusters_best = dbscan_best.fit_predict(X_test_DBSCAN_Original)
anomalies_DBSCAN_Original = (clusters_best == -1)  # Mark clusters labeled -1 as anomalies
test_data_DBSCAN_Original = test_data.copy()

# Create a binary label for DBSCAN-detected anomalies (1: anomaly, 0: normal)
test_data_DBSCAN_Original['DBSCAN_Anomaly'] = anomalies_DBSCAN_Original.astype(int)

# Model's predictions on the testing data using DBSCAN
y_pred_DBSCAN_Original = test_data_DBSCAN_Original['DBSCAN_Anomaly']

# Evaluate the performance of DBSCAN
accuracy_DBSCAN_Original = accuracy_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
precision_DBSCAN_Original = precision_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
recall_DBSCAN_Original = recall_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
f1_DBSCAN_Original = f1_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)

# Confusion matrix
conf_matrix_DBSCAN_Original = confusion_matrix(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
print("Confusion Matrix:")
print(conf_matrix_DBSCAN_Original)

# Calculate specificity
tn, fp, fn, tp = conf_matrix_DBSCAN_Original.ravel()
specificity_DBSCAN_Original = tn / (tn + fp)

# Calculate AUC
auc_DBSCAN_Original = roc_auc_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)

# Print DBSCAN performance metrics
print("DBSCAN Performance:")
print("Accuracy:", accuracy_DBSCAN_Original)
print("Precision:", precision_DBSCAN_Original)
print("Recall:", recall_DBSCAN_Original)
print("F1-score:", f1_DBSCAN_Original)
print("Specificity:", specificity_DBSCAN_Original)
print("AUC:", auc_DBSCAN_Original)

In [ ]:
plot_movement_with_anomalies(
    data=data,
    test_data=test_data_DBSCAN_Original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    model_name='DBSCAN',
    anomaly_column='Anomaly',
    prediction_column='DBSCAN_Anomaly'
)

In [ ]:
plot_movement_with_anomalies2020(
    data=data,
    test_data=test_data_DBSCAN_Original,
    start_date='2020-01-01',
    end_date='2020-12-31',
    model_name='DBSCAN',
    anomaly_column='Anomaly',
    prediction_column='DBSCAN_Anomaly'
)

In [ ]:
# DBSCAN
plot_movement_with_anomalies_only(
    data=data,
    test_data=test_data_DBSCAN_Original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    specific_fishid= 'A69-9001-23859a',
    model_name='DBSCAN',
    anomaly_column='Anomaly',
    prediction_column='DBSCAN_Anomaly'
)

# LOF Plots

In [ ]:
# Ground truth labels (actual anomalies) from the testing set
y_true_LOF_original = test_data['Anomaly']

# Feature selection for LOF
selected_features_LOF_original = ['receiver', 'lat', 'lon', 'duration_in_same_station', 'num_detections',
                                  'distance', 'num_days_detected', 'num_unique_stations', 'consecutive_missing_stations']

X_train_LOF_original = training_data1[selected_features_LOF_original]
X_test_LOF_original = test_data[selected_features_LOF_original]

# Use the best parameters to create the LOF model
lof_best = LocalOutlierFactor(n_neighbors=loaded_best_params_LOF_Original['n_neighbors'],

                              contamination=loaded_best_params_LOF_Original['contamination'])

# Predict anomalies with LOF
anomalies_best = lof_best.fit_predict(X_test_LOF_original)
anomalies_LOF_original = (anomalies_best == -1)  # Mark LOF detected anomalies (outliers) as 1

# Create test_data_LOF_original similar to test_data_5
test_data_LOF_original = test_data.copy()  # Copy original test data
test_data_LOF_original['LOF_Anomaly'] = anomalies_LOF_original.astype(int)  # Add LOF anomalies as a new column

# Model's predictions on the testing data using LOF
y_pred_LOF_original = test_data_LOF_original['LOF_Anomaly']

# Evaluate the performance of LOF
accuracy_LOF_original = accuracy_score(y_true_LOF_original, y_pred_LOF_original)
precision_LOF_original = precision_score(y_true_LOF_original, y_pred_LOF_original)
recall_LOF_original = recall_score(y_true_LOF_original, y_pred_LOF_original)
f1_LOF_original = f1_score(y_true_LOF_original, y_pred_LOF_original)

# Confusion matrix
conf_matrix_LOF_original = confusion_matrix(y_true_LOF_original, y_pred_LOF_original)
print("Confusion Matrix:")
print(conf_matrix_LOF_original)

# Calculate specificity
tn, fp, fn, tp = conf_matrix_LOF_original.ravel()
specificity_LOF_original = tn / (tn + fp)
print("LOF Performance:")
print("Accuracy:", accuracy_LOF_original)
print("Precision:", precision_LOF_original)
print("Recall:", recall_LOF_original)
print("F1-score:", f1_LOF_original)
print("Specificity:", specificity_LOF_original)

In [ ]:
plot_movement_with_anomalies(
    data=data,
    test_data=test_data_LOF_original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    model_name='LOF',
    anomaly_column='Anomaly',
    prediction_column='LOF_Anomaly'
)

In [ ]:
plot_movement_with_anomalies2020(
    data=data,
    test_data=test_data_LOF_original,
    start_date='2020-01-01',
    end_date='2020-12-31',
    model_name='LOF',
    anomaly_column='Anomaly',
    prediction_column='LOF_Anomaly'
)

In [ ]:
# Local Outlier Factor (LOF)
plot_movement_with_anomalies_only(
    data=data,
    test_data=test_data_LOF_original,
    start_date='2016-01-01',
    end_date='2021-12-31',
    specific_fishid= 'A69-9001-23859a',
    model_name='LOF',
    anomaly_column='Anomaly',
    prediction_column='LOF_Anomaly'
)

In [ ]:
# Define confusion matrices
conf_matrix_isolation_Forest = np.array([[274996, 0], [131843, 142]])
conf_matrix_DBSCAN = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE = np.array([[274031, 965], [0, 131985]])

# Define additional confusion matrices with _90 and _65
conf_matrix_isolation_Forest_90 = np.array([[274996, 0], [131784, 201]])
conf_matrix_DBSCAN_90 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_90 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_90 = np.array([[274077, 919], [0, 131985]])

conf_matrix_isolation_Forest_65 = np.array([[274996, 0], [131764, 221]])
conf_matrix_DBSCAN_65 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_65 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_65 = np.array([[274394, 602], [0, 131985]])

# Function to calculate metrics from confusion matrix
def calculate_metrics(conf_matrix):
    TN, FP, FN, TP = conf_matrix.ravel()
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  # True Positive Rate (Recall)
    TNR = TN / (TN + FP) if (TN + FP) > 0 else 0  # True Negative Rate
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # False Positive Rate
    FNR = FN / (TP + FN) if (TP + FN) > 0 else 0  # False Negative Rate
    return [TPR * 100, TNR * 100, FPR * 100, FNR * 100]  # List format

# Calculate metrics for each model and for the new confusion matrices
metrics_NN_AE = calculate_metrics(conf_matrix_NN_AE)
metrics_isolation_Forest = calculate_metrics(conf_matrix_isolation_Forest)
metrics_DBSCAN = calculate_metrics(conf_matrix_DBSCAN)
metrics_Local_Outlier_Factor = calculate_metrics(conf_matrix_Local_Outlier_Factor)

metrics_NN_AE_90 = calculate_metrics(conf_matrix_NN_AE_90)
metrics_isolation_Forest_90 = calculate_metrics(conf_matrix_isolation_Forest_90)
metrics_DBSCAN_90 = calculate_metrics(conf_matrix_DBSCAN_90)
metrics_Local_Outlier_Factor_90 = calculate_metrics(conf_matrix_Local_Outlier_Factor_90)

metrics_NN_AE_65 = calculate_metrics(conf_matrix_NN_AE_65)
metrics_isolation_Forest_65 = calculate_metrics(conf_matrix_isolation_Forest_65)
metrics_DBSCAN_65 = calculate_metrics(conf_matrix_DBSCAN_65)
metrics_Local_Outlier_Factor_65 = calculate_metrics(conf_matrix_Local_Outlier_Factor_65)

# Collect metrics in a dictionary
model_labels = ['NN-AE', 'IF', 'DBSCAN', 'LOF']
metrics_dict = {
    'NN-AE': [metrics_NN_AE, metrics_NN_AE_90, metrics_NN_AE_65],
    'IF': [metrics_isolation_Forest, metrics_isolation_Forest_90, metrics_isolation_Forest_65],
    'DBSCAN': [metrics_DBSCAN, metrics_DBSCAN_90, metrics_DBSCAN_65],
    'LOF': [metrics_Local_Outlier_Factor, metrics_Local_Outlier_Factor_90, metrics_Local_Outlier_Factor_65]
}

# Define metric names and bar plot settings
metrics_names = ['True Anomaly Rate', 'True Normal Rate', 'False Anomaly Rate', 'False Normal Rate']
index = np.arange(len(metrics_names))
bar_width = 0.2


# Create a 3x1 grid of subplots (first two on top, third one at the bottom)
fig, axes = plt.subplots(3, 1, figsize=(18, 10))

# Subplots: Bar plots for each subplot (first two on top, third one at the bottom)
for i, model in enumerate(model_labels):
    for j in range(2):  # Iterate over the first two confusion matrices (original, 90)
        metrics = metrics_dict[model][j]
        if j == 0:
            bars = axes[0].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)
        else:
            bars = axes[1].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

        # Add percentage labels on top of the bars
        for bar in bars:
            height = bar.get_height()
            if j == 0:
                axes[0].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)
            else:
                axes[1].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# For the third subplot (bottom one), use the 65% confusion matrices
for i, model in enumerate(model_labels):
    metrics = metrics_dict[model][2]
    bars = axes[2].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

    # Add percentage labels on top of the bars
    for bar in bars:
        height = bar.get_height()
        axes[2].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# Set common settings for all subplots
ii=0
for ax in axes.flat:
    if ii==0:
      ax.legend(loc='best', fontsize=20, bbox_to_anchor=(1.00, 0.5))  # Add legend to each subplot
      ii=ii+1
    ax.set_xticks(index)
    ax.set_xticklabels(metrics_names, fontsize=20)
    ax.set_ylabel('Percentage',fontsize=20)
    ax.set_ylim(0, 130)  # Set y-axis limit from 0 to 130
    ax.yaxis.set_tick_params(labelsize=15)



# Set titles for each subplot
axes[0].set_title('Without resampling', fontsize=20)
axes[1].set_title('Resampling 90s', fontsize=20)
axes[2].set_title('Resampling 65s', fontsize=20)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
#confusion matrices for each model
conf_matrix_NN_AE = confusion_matrix(y_true, y_pred)
conf_matrix_isolation_Forest = confusion_matrix(y_true_model_IF_original, y_pred_model_IF_original)
conf_matrix_DBSCAN = confusion_matrix(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
conf_matrix_Local_Outlier_Factor = confusion_matrix(y_true_LOF_original, y_pred_LOF_original)

# Function to calculate metrics from confusion matrix
def calculate_metrics(conf_matrix):
    TN, FP, FN, TP = conf_matrix.ravel()
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  # True Positive Rate (Recall)
    TNR = TN / (TN + FP) if (TN + FP) > 0 else 0  # True Negative Rate
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # False Positive Rate
    FNR = FN / (TP + FN) if (TP + FN) > 0 else 0  # False Negative Rate
    return TPR * 100, TNR * 100, FPR * 100, FNR * 100  # Convert to percentage

# Calculate metrics for each model
metrics_NN_AE = calculate_metrics(conf_matrix_NN_AE)
metrics_isolation_Forest = calculate_metrics(conf_matrix_isolation_Forest)
metrics_DBSCAN = calculate_metrics(conf_matrix_DBSCAN)
metrics_Local_Outlier_Factor = calculate_metrics(conf_matrix_Local_Outlier_Factor)

# Collect all metrics in the specified model order
model_labels = ['NN-AE', 'IF', 'DBSCAN', 'LOF']
metrics_dict = {
    'NN-AE': metrics_NN_AE,
    'IF': metrics_isolation_Forest,
    'DBSCAN': metrics_DBSCAN,
    'LOF': metrics_Local_Outlier_Factor
}

# Define metrics names and positions
metrics_names = ['True Anomaly Rate', 'True Normal Rate', 'False Anolay Rate', 'False Normal Rate']
index = np.arange(len(metrics_names))
bar_width = 0.2

# Create the bar plots
plt.figure(figsize=(14, 8))

for i, model in enumerate(model_labels):
    model_metrics = [metrics_dict[model][j] for j in range(len(metrics_names))]
    plt.bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2,
            model_metrics,
            bar_width,
            label=model)

# Function to add labels on top of each bar
def add_labels(index, metrics_names, metrics_dict, model_labels):
    for i, model in enumerate(model_labels):
        for j, metric_name in enumerate(metrics_names):
            plt.text(index[j] + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2,
                     metrics_dict[model][j] + 1,
                     f'{metrics_dict[model][j]:.1f}%',
                     ha='center', va='bottom')

# Adding labels to each bar
add_labels(index, metrics_names, metrics_dict, model_labels)

# Customize the plot
plt.xlabel('Metrics')
plt.ylabel('Percentage')
plt.title('Comparison of Confusion Matrix Metrics Across Different Models')
plt.xticks(index, metrics_names)
plt.ylim(0, 120)
plt.legend(loc='upper right')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
conf_matrix_NN_AE = confusion_matrix(y_true, y_pred)
conf_matrix_isolation_Forest = confusion_matrix(y_true_model_IF_original, y_pred_model_IF_original)
conf_matrix_DBSCAN = confusion_matrix(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
conf_matrix_Local_Outlier_Factor = confusion_matrix(y_true_LOF_original, y_pred_LOF_original)
# Extracting values from the confusion matrix
TNs, FPs, FNs, TPs = [], [], [], []
for conf_matrix in [conf_matrix_isolation_Forest,conf_matrix_DBSCAN ,conf_matrix_Local_Outlier_Factor ,conf_matrix_NN_AE]:
    TN, FP, FN, TP = conf_matrix.ravel()
    TNs.append(TN)
    FPs.append(FP)
    FNs.append(FN)
    TPs.append(TP)

# Define model names
model_names = ['IF', 'DBSCAN','LOF','NN-AE']

# Calculate the percentages for each model
true_negative_rates = [(TN / (TN + FP)) * 100 for TN, FP in zip(TNs, FPs)]
false_positive_rates = [(FP / (TN + FP)) * 100 for TN, FP in zip(TNs, FPs)]
true_positive_rates = [(TP / (TP + FN)) * 100 for TP, FN in zip(TPs, FNs)]
false_negative_rates = [(FN / (TP + FN)) * 100 for TP, FN in zip(TPs, FNs)]

# Plotting
plt.figure(figsize=(15, 8))

# Number of models
num_models = len(true_negative_rates)

# Plot bars for each rate
bar_width = 0.2
index = np.arange(num_models)

# Function to add labels on top of each bar
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.annotate(f'{height:.0f}%',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')

tn_bars = plt.bar(index, true_negative_rates, bar_width, label='TN rate', color='blue', alpha=0.7)
fp_bars = plt.bar(index + bar_width, false_positive_rates, bar_width, label='FA rate', color='green', alpha=0.7)
tp_bars = plt.bar(index + 2 * bar_width, true_positive_rates, bar_width, label='TA rate', color='red', alpha=0.7)
fn_bars = plt.bar(index + 3 * bar_width, false_negative_rates, bar_width, label='FN Rate', color='purple', alpha=0.7)

# Adding labels to each bar
add_labels(tn_bars)
add_labels(fp_bars)
add_labels(tp_bars)
add_labels(fn_bars)

plt.xlabel('Models')
plt.ylabel('Percentage')
plt.title('Rates by Model')
plt.xticks(index + 1.5 * bar_width, model_names)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Plot ROC curve for each model
plt.figure(figsize=(8, 6))

fpr_NN_AE, tpr_NN_AE, _ = roc_curve(y_true, y_pred)
auc_NN_AE = auc(fpr_NN_AE, tpr_NN_AE)
plt.plot(fpr_NN_AE, tpr_NN_AE, label=f'NN-AE (AUC = {auc_NN_AE:.2f})')

fpr_IF, tpr_IF, _ = roc_curve(y_true_model_IF_original, y_pred_model_IF_original)
auc_IF = auc(fpr_IF, tpr_IF)
plt.plot(fpr_IF, tpr_IF, label=f'IF (AUC = {auc_IF:.2f})')

fpr_DBSCAN, tpr_DBSCAN, _ = roc_curve(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
auc_DBSCAN = auc(fpr_DBSCAN, tpr_DBSCAN)
plt.plot(fpr_DBSCAN, tpr_DBSCAN, label=f'DBSCAN (AUC = {auc_DBSCAN:.2f})')

fpr_LOF, tpr_LOF, _ = roc_curve(y_true_LOF_original, y_pred_LOF_original)
auc_LOF = auc(fpr_LOF, tpr_LOF)
plt.plot(fpr_LOF, tpr_LOF, label=f'LOF (AUC = {auc_LOF:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Calculate accuracy, precision, recall, and F1-score for each model
accuracy_NN_AE = accuracy_score(y_true, y_pred)
precision_NN_AE = precision_score(y_true, y_pred)
recall_NN_AE = recall_score(y_true, y_pred)
f1_NN_AE = f1_score(y_true, y_pred)

accuracy_IF = accuracy_score(y_true_model_IF_original, y_pred_model_IF_original)
precision_IF = precision_score(y_true_model_IF_original, y_pred_model_IF_original)
recall_IF = recall_score(y_true_model_IF_original, y_pred_model_IF_original)
f1_IF = f1_score(y_true_model_IF_original, y_pred_model_IF_original)

accuracy_DBSCAN = accuracy_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
precision_DBSCAN = precision_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
recall_DBSCAN = recall_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)
f1_DBSCAN = f1_score(y_true_DBSCAN_Original, y_pred_DBSCAN_Original)

accuracy_LOF = accuracy_score(y_true_LOF_original, y_pred_LOF_original)
precision_LOF = precision_score(y_true_LOF_original, y_pred_LOF_original)
recall_LOF = recall_score(y_true_LOF_original, y_pred_LOF_original)
f1_LOF = f1_score(y_true_LOF_original, y_pred_LOF_original)

# Collect all metrics in the specified model order
model_labels = ['NN-AE', 'IF', 'DBSCAN', 'LOF']
metrics_dict = {
    'NN-AE': [accuracy_NN_AE, precision_NN_AE, recall_NN_AE, f1_NN_AE],
    'IF': [accuracy_IF, precision_IF, recall_IF, f1_IF],
    'DBSCAN': [accuracy_DBSCAN, precision_DBSCAN, recall_DBSCAN, f1_DBSCAN],
    'LOF': [accuracy_LOF, precision_LOF, recall_LOF, f1_LOF]
}

# Define metrics names and positions
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
index = np.arange(len(metrics_names))
bar_width = 0.2

# Create the bar plots
plt.figure(figsize=(14, 8))
for i, model in enumerate(model_labels):
    model_metrics = [metrics_dict[model][j] for j in range(len(metrics_names))]
    plt.bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, model_metrics, bar_width, label=model)

# Function to add labels on top of each bar
def add_labels(index, metrics_names, metrics_dict, model_labels):
    for i, model in enumerate(model_labels):
        for j, metric_name in enumerate(metrics_names):
            plt.text(index[j] + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics_dict[model][j] + 0.01, f'{metrics_dict[model][j]:.2f}', ha='center', va='bottom')

# Adding labels to each bar
add_labels(index, metrics_names, metrics_dict, model_labels)

# Customize the plot
plt.xlabel('Metrics')
plt.ylabel('Percentage')
plt.title('Comparison of Model Performance Metrics')
plt.xticks(index, metrics_names)
plt.ylim(0, 1.1)
plt.legend(loc='upper right')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define confusion matrices
conf_matrix_isolation_Forest = np.array([[274996, 0], [131843, 142]])
conf_matrix_DBSCAN = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE = np.array([[274031, 965], [0, 131985]])

# Define additional confusion matrices with _90 and _65
conf_matrix_isolation_Forest_90 = np.array([[274996, 0], [131784, 201]])
conf_matrix_DBSCAN_90 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_90 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_90 = np.array([[274077, 919], [0, 131985]])

conf_matrix_isolation_Forest_65 = np.array([[274996, 0], [131764, 221]])
conf_matrix_DBSCAN_65 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_65 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_65 = np.array([[274394, 602], [0, 131985]])

# Function to calculate metrics from confusion matrix
def calculate_metrics(conf_matrix):
    TN, FP, FN, TP = conf_matrix.ravel()
    accuracy = (TP + TN) / (TP + TN + FP + FN) * 100 if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) * 100 if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) * 100 if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return [accuracy, precision, recall, f1_score]

# Calculate metrics for each model and for the new confusion matrices
metrics_NN_AE = calculate_metrics(conf_matrix_NN_AE)
metrics_isolation_Forest = calculate_metrics(conf_matrix_isolation_Forest)
metrics_DBSCAN = calculate_metrics(conf_matrix_DBSCAN)
metrics_Local_Outlier_Factor = calculate_metrics(conf_matrix_Local_Outlier_Factor)

metrics_NN_AE_90 = calculate_metrics(conf_matrix_NN_AE_90)
metrics_isolation_Forest_90 = calculate_metrics(conf_matrix_isolation_Forest_90)
metrics_DBSCAN_90 = calculate_metrics(conf_matrix_DBSCAN_90)
metrics_Local_Outlier_Factor_90 = calculate_metrics(conf_matrix_Local_Outlier_Factor_90)

metrics_NN_AE_65 = calculate_metrics(conf_matrix_NN_AE_65)
metrics_isolation_Forest_65 = calculate_metrics(conf_matrix_isolation_Forest_65)
metrics_DBSCAN_65 = calculate_metrics(conf_matrix_DBSCAN_65)
metrics_Local_Outlier_Factor_65 = calculate_metrics(conf_matrix_Local_Outlier_Factor_65)

# Collect metrics in a dictionary
model_labels = ['NN-AE', 'IF', 'DBSCAN', 'LOF']
metrics_dict = {
    'NN-AE': [metrics_NN_AE, metrics_NN_AE_90, metrics_NN_AE_65],
    'IF': [metrics_isolation_Forest, metrics_isolation_Forest_90, metrics_isolation_Forest_65],
    'DBSCAN': [metrics_DBSCAN, metrics_DBSCAN_90, metrics_DBSCAN_65],
    'LOF': [metrics_Local_Outlier_Factor, metrics_Local_Outlier_Factor_90, metrics_Local_Outlier_Factor_65]
}

# Define metric names and bar plot settings
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
index = np.arange(len(metrics_names))
bar_width = 0.2

# Create a 3x1 grid of subplots
fig, axes = plt.subplots(3, 1, figsize=(18, 10))

# Subplots: Bar plots for each subplot
for i, model in enumerate(model_labels):
    for j in range(2):  # Iterate over the first two confusion matrices (original, 90)
        metrics = metrics_dict[model][j]
        if j == 0:
            bars = axes[0].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)
        else:
            bars = axes[1].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

        # Add percentage labels on top of the bars
        for bar in bars:
            height = bar.get_height()
            if j == 0:
                axes[0].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)
            else:
                axes[1].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# For the third subplot (bottom one), use the 65% confusion matrices
for i, model in enumerate(model_labels):
    metrics = metrics_dict[model][2]
    bars = axes[2].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

    # Add percentage labels on top of the bars
    for bar in bars:
        height = bar.get_height()
        axes[2].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# Set common settings for all subplots
ii = 0
for ax in axes.flat:
    if ii == 0:
        ax.legend(loc='best', fontsize=20, bbox_to_anchor=(1.00, 0.5))
        ii = ii + 1
    ax.set_xticks(index)
    ax.set_xticklabels(metrics_names, fontsize=20)
    ax.set_ylabel('Percentage', fontsize=20)
    ax.set_ylim(0, 130)  # Set y-axis limit from 0 to 130
    ax.yaxis.set_tick_params(labelsize=15)

# Set titles for each subplot
axes[0].set_title('Without resampling', fontsize=20)
axes[1].set_title('Resampling 90s', fontsize=20)
axes[2].set_title('Resampling 65s', fontsize=20)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define confusion matrices
conf_matrix_isolation_Forest = np.array([[274996, 0], [131843, 142]])
conf_matrix_DBSCAN = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE = np.array([[274031, 965], [0, 131985]])

# Define additional confusion matrices with _90 and _65
conf_matrix_isolation_Forest_90 = np.array([[274996, 0], [131784, 201]])
conf_matrix_DBSCAN_90 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_90 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_90 = np.array([[274077, 919], [0, 131985]])

conf_matrix_isolation_Forest_65 = np.array([[274996, 0], [131764, 221]])
conf_matrix_DBSCAN_65 = np.array([[268314, 6682], [131656, 329]])
conf_matrix_Local_Outlier_Factor_65 = np.array([[271462, 3534], [131449, 536]])
conf_matrix_NN_AE_65 = np.array([[274394, 602], [0, 131985]])

# Function to calculate metrics from confusion matrix
def calculate_metrics(conf_matrix, auc=0):
    TN, FP, FN, TP = conf_matrix.ravel()
    accuracy = (TP + TN) / (TP + TN + FP + FN) * 100 if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) * 100 if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) * 100 if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return [accuracy, precision, recall, f1_score, auc]

# Simulate AUC values for demonstration purposes
simulated_aucs = {
    'NN-AE': [99.82, 99.83, 99.89],
    'IF': [50.05, 50.07, 50.08],
    'DBSCAN': [48.90, 48.90, 48.90],
    'LOF': [49.56, 49.56, 49.56]
}

# Calculate metrics for each model and for the new confusion matrices
metrics_NN_AE = calculate_metrics(conf_matrix_NN_AE, simulated_aucs['NN-AE'][0])
metrics_isolation_Forest = calculate_metrics(conf_matrix_isolation_Forest, simulated_aucs['IF'][0])
metrics_DBSCAN = calculate_metrics(conf_matrix_DBSCAN, simulated_aucs['DBSCAN'][0])
metrics_Local_Outlier_Factor = calculate_metrics(conf_matrix_Local_Outlier_Factor, simulated_aucs['LOF'][0])

metrics_NN_AE_90 = calculate_metrics(conf_matrix_NN_AE_90, simulated_aucs['NN-AE'][1])
metrics_isolation_Forest_90 = calculate_metrics(conf_matrix_isolation_Forest_90, simulated_aucs['IF'][1])
metrics_DBSCAN_90 = calculate_metrics(conf_matrix_DBSCAN_90, simulated_aucs['DBSCAN'][1])
metrics_Local_Outlier_Factor_90 = calculate_metrics(conf_matrix_Local_Outlier_Factor_90, simulated_aucs['LOF'][1])

metrics_NN_AE_65 = calculate_metrics(conf_matrix_NN_AE_65, simulated_aucs['NN-AE'][2])
metrics_isolation_Forest_65 = calculate_metrics(conf_matrix_isolation_Forest_65, simulated_aucs['IF'][2])
metrics_DBSCAN_65 = calculate_metrics(conf_matrix_DBSCAN_65, simulated_aucs['DBSCAN'][2])
metrics_Local_Outlier_Factor_65 = calculate_metrics(conf_matrix_Local_Outlier_Factor_65, simulated_aucs['LOF'][2])

# Collect metrics in a dictionary
model_labels = ['NN-AE', 'IF', 'DBSCAN', 'LOF']
metrics_dict = {
    'NN-AE': [metrics_NN_AE, metrics_NN_AE_90, metrics_NN_AE_65],
    'IF': [metrics_isolation_Forest, metrics_isolation_Forest_90, metrics_isolation_Forest_65],
    'DBSCAN': [metrics_DBSCAN, metrics_DBSCAN_90, metrics_DBSCAN_65],
    'LOF': [metrics_Local_Outlier_Factor, metrics_Local_Outlier_Factor_90, metrics_Local_Outlier_Factor_65]
}

# Define metric names and bar plot settings
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC']
index = np.arange(len(metrics_names))
bar_width = 0.2

# Create a 3x1 grid of subplots
fig, axes = plt.subplots(3, 1, figsize=(18, 10))

# Subplots: Bar plots for each subplot
for i, model in enumerate(model_labels):
    for j in range(2):  # Iterate over the first two confusion matrices (original, 90)
        metrics = metrics_dict[model][j]
        if j == 0:
            bars = axes[0].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)
        else:
            bars = axes[1].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

        # Add percentage labels on top of the bars
        for bar in bars:
            height = bar.get_height()
            if j == 0:
                axes[0].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)
            else:
                axes[1].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# For the third subplot (bottom one), use the 65% confusion matrices
for i, model in enumerate(model_labels):
    metrics = metrics_dict[model][2]
    bars = axes[2].bar(index + i * bar_width - (bar_width * (len(model_labels) - 1)) / 2, metrics, bar_width, label=model)

    # Add percentage labels on top of the bars
    for bar in bars:
        height = bar.get_height()
        axes[2].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f}', ha='center', va='bottom', fontsize=15)

# Set common settings for all subplots
ii = 0
for ax in axes.flat:
    if ii == 0:
        ax.legend(loc='best', fontsize=20, bbox_to_anchor=(1.00, 0.5))
        ii = ii + 1
    ax.set_xticks(index)
    ax.set_xticklabels(metrics_names, fontsize=20)
    ax.set_ylabel('Percentage', fontsize=20)
    ax.set_ylim(0, 130)  # Set y-axis limit from 0 to 130
    ax.yaxis.set_tick_params(labelsize=15)

# Set titles for each subplot
axes[0].set_title('Without resampling', fontsize=20)
axes[1].set_title('Resampling 90s', fontsize=20)
axes[2].set_title('Resampling 65s', fontsize=20)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Sample data
models = ['IF: without resampling ', 'IF: resampling 90s', 'IF: resampling 65s', 'DBSCAN: without resampling', 'DBSCAN: resampling 90s', 'DBSCAN: resampling 65s','LOF: without resampling', 'LOF: resampling 90s', 'LOF: resampling 65s', 'NN-AE without resampling', 'NN-AE: resampling 90s', 'NN-AE: resampling 65s']
accuracies = [0.6760,0.6762,0.6761, 0.6600, 0.6600, 0.6600, 0.6683, 0.6683, 0.6683, 0.9976, 0.9977,0.9985]
f1_scores = [0.0021, 0.0033, 0.0030, 0.0047, 0.0047,0.0047, 0.0078, 0.0078,  0.0078, 0.9963, 0.9980,0.9977]
recalls = [0.0010, 0.0016, 0.0015, 0.0024, 0.0024, 0.0024, 0.0040, 0.0040,  0.0040, 1.0, 1.0, 1.0]
precisions = [1.0, 1.0, 1.0, 0.0469, 0.0469, 0.0469, 0.1316, 0.1316, 0.1316, 0.9927, 0.9861,0.9854]
specificities = [1.0, 1.0, 1.0, 0.9757,0.9757, 0.9757, 0.9871, 0.9871, 0.9871, 0.9965, 0.9981,0.9978]
auc_scores = [0.5005, 0.5008,  0.5007, 0.4890, 0.4890, 0.4890, 0.4956, 0.4956, 0.4956, 0.9983, 0.9990,0.9989]

# Confidence intervals for each metric
ci_acc = [0.0011, 0.0012, 0.0012, 0.0011, 0.0012, 0.0012, 0.0013, 0.0012, 0.0013, 0.0013, 0.0002, 0.0002]
ci_f1 = [0.0003, 0.0004, 0.0003, 0.0004, 0.0004, 0.0005, 0.0006, 0.0005, 0.0006, 0.0006, 0.0003, 0.0002]
ci_recall = [0.0001, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0003, 0.0003, 0.0003, 0.0003, 0, 0]
ci_precision = [0, 0, 0,0.0036, 0.0040, 0.0040, 0.0092, 0.0091, 0.0084, 0.0092, 0.0006, 0.0005]
ci_specificity = [0, 0, 0, 0.0004, 0.0005, 0.0004, 0.0004, 0.0003, 0.0003, 0.0002, 0.0003, 0.0003]
ci_auc = [0.0001, 0.0001, 0.0001, 0.0003, 0.0003, 0.0002, 0.0003, 0.0002, 0.0002, 0.0001, 0.0001, 0.0001]

# Colors for different metrics
colors = {
    'Accuracy': 'red',
    'F1-Score': 'blue',
    'Recall': 'green',
    'Precision': 'purple',
    'Specificity': 'orange',
    'AUC': 'brown'
}

# Create plot
fig, ax = plt.subplots(figsize=(14, 14))

# Plot each model with its metrics and confidence intervals
for i in range(len(models)):
    base_pos = i * 7
    ax.errorbar(accuracies[i], base_pos + 5, xerr=ci_acc[i], fmt='o', color=colors['Accuracy'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(f1_scores[i], base_pos + 4, xerr=ci_f1[i], fmt='o', color=colors['F1-Score'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(recalls[i], base_pos + 3, xerr=ci_recall[i], fmt='o', color=colors['Recall'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(precisions[i], base_pos + 2, xerr=ci_precision[i], fmt='o', color=colors['Precision'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(specificities[i], base_pos + 1, xerr=ci_specificity[i], fmt='o', color=colors['Specificity'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(auc_scores[i], base_pos, xerr=ci_auc[i], fmt='o', color=colors['AUC'], capsize=2, capthick=1, markersize=4)

# Customizing the plot
ax.set_yticks([i * 7 + 2.5 for i in range(len(models))])
ax.set_yticklabels(models, fontsize=20)
ax.set_xticks(np.arange(0, 1.1, 0.1))
ax.set_xlabel('Percentage [x100%]', fontsize=20)
ax.set_ylabel('', fontsize=20)
ax.grid(True, linestyle='--', alpha=0.7, axis='x')
ax.xaxis.set_tick_params(labelsize=15)

# Adding horizontal lines to separate models
for i in range(1, len(models)):
    ax.axhline(y=i * 7 - 0.5, color='black', linestyle='--', linewidth=1)

# Add a legend outside the plot
legend_handles = [
    plt.Line2D([0], [0], color=colors['Accuracy'], lw=2, label='Accuracy'),
    plt.Line2D([0], [0], color=colors['F1-Score'], lw=2, label='F1-Score'),
    plt.Line2D([0], [0], color=colors['Recall'], lw=2, label='Recall'),
    plt.Line2D([0], [0], color=colors['Precision'], lw=2, label='Precision'),
    plt.Line2D([0], [0], color=colors['Specificity'], lw=2, label='Specificity'),
    plt.Line2D([0], [0], color=colors['AUC'], lw=2, label='AUC')
]
ax.legend(handles=legend_handles, loc='best', bbox_to_anchor=(1, 0.5), fontsize=20, frameon=False)

# plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

plt.show()

In [ ]:
# Updated order for models
models = [
    'DBSCAN: resampling 65s', 'LOF: resampling 65s', 'IF: resampling 65s', 'NN-AE: resampling 65s',
    'DBSCAN: resampling 90s', 'LOF: resampling 90s', 'IF: resampling 90s', 'NN-AE: resampling 90s',
    'DBSCAN: without resampling', 'LOF: without resampling', 'IF: without resampling', 'NN-AE: without resampling'
]

# Metrics updated in the specified order
accuracies = [0.6600, 0.6683, 0.6761, 0.9985, 0.6600, 0.6683, 0.6762, 0.9977, 0.6600, 0.6683, 0.6760, 0.9976]
f1_scores = [0.0047, 0.0078, 0.0030, 0.9977, 0.0047, 0.0078, 0.0033, 0.9980, 0.0047, 0.0078, 0.0021, 0.9963]
recalls = [0.0024, 0.0040, 0.0015, 1.0, 0.0024, 0.0040, 0.0016, 1.0, 0.0024, 0.0040, 0.0010, 1.0]
precisions = [0.0469, 0.1316, 1.0, 0.9854, 0.0469, 0.1316, 1.0, 0.9861, 0.0469, 0.1316, 1.0, 0.9927]
specificities = [0.9757, 0.9871, 1.0, 0.9978, 0.9757, 0.9871, 1.0, 0.9981, 0.9757, 0.9871, 1.0, 0.9965]
auc_scores = [0.4890, 0.4956, 0.5007, 0.9989, 0.4890, 0.4956, 0.5008, 0.9990, 0.4890, 0.4956, 0.5005, 0.9983]

# Confidence intervals for each metric (matching the new order)
ci_acc = [0.0012, 0.0013, 0.0012, 0.0002, 0.0012, 0.0012, 0.0012, 0.0002, 0.0011, 0.0013, 0.0011, 0.0013]
ci_f1 = [0.0004, 0.0006, 0.0003, 0.0002, 0.0004, 0.0005, 0.0004, 0.0003, 0.0004, 0.0006, 0.0003, 0.0006]
ci_recall = [0.0002, 0.0003, 0.0002, 0, 0.0002, 0.0003, 0.0002, 0, 0.0002, 0.0003, 0.0001, 0.0003]
ci_precision = [0.0040, 0.0084, 0, 0.0005, 0.0040, 0.0091, 0, 0.0006, 0.0036, 0.0092, 0, 0.0092]
ci_specificity = [0.0004, 0.0003, 0, 0.0003, 0.0005, 0.0003, 0, 0.0003, 0.0004, 0.0003, 0, 0.0002]
ci_auc = [0.0003, 0.0002, 0.0001, 0.0001, 0.0003, 0.0002, 0.0001, 0.0001, 0.0003, 0.0002, 0.0001, 0.0001]

# Colors for different metrics
colors = {
    'Accuracy': 'red',
    'F1-Score': 'blue',
    'Recall': 'green',
    'Precision': 'purple',
    'Specificity': 'orange',
    'AUC': 'brown'
}

# Create plot
fig, ax = plt.subplots(figsize=(14, 14))

# Plot each model with its metrics and confidence intervals
for i in range(len(models)):
    base_pos = i * 7
    ax.errorbar(accuracies[i], base_pos + 5, xerr=ci_acc[i], fmt='o', color=colors['Accuracy'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(f1_scores[i], base_pos + 4, xerr=ci_f1[i], fmt='o', color=colors['F1-Score'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(recalls[i], base_pos + 3, xerr=ci_recall[i], fmt='o', color=colors['Recall'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(precisions[i], base_pos + 2, xerr=ci_precision[i], fmt='o', color=colors['Precision'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(specificities[i], base_pos + 1, xerr=ci_specificity[i], fmt='o', color=colors['Specificity'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(auc_scores[i], base_pos, xerr=ci_auc[i], fmt='o', color=colors['AUC'], capsize=2, capthick=1, markersize=4)

# Customizing the plot
ax.set_yticks([i * 7 + 2.5 for i in range(len(models))])
ax.set_yticklabels(models, fontsize=20)
ax.set_xticks(np.arange(0, 1.1, 0.1))
ax.set_xlabel('Percentage [x100%]', fontsize=20)
ax.set_ylabel('', fontsize=20)
ax.grid(True, linestyle='--', alpha=0.7, axis='x')
ax.xaxis.set_tick_params(labelsize=15)

# Adding horizontal lines to separate models
# Adding horizontal lines to separate models
for i in range(1, len(models)):
    if i == 4 or i == 8:
        ax.axhline(y=i * 7 - 0.5, color='black', linestyle='-', linewidth=2)  # Bold line
    else:
        ax.axhline(y=i * 7 - 0.5, color='black', linestyle='--', linewidth=1)

# Add a legend outside the plot
legend_handles = [
    plt.Line2D([0], [0], color=colors['Accuracy'], lw=2, label='Accuracy'),
    plt.Line2D([0], [0], color=colors['F1-Score'], lw=2, label='F1-Score'),
    plt.Line2D([0], [0], color=colors['Recall'], lw=2, label='Recall'),
    plt.Line2D([0], [0], color=colors['Precision'], lw=2, label='Precision'),
    plt.Line2D([0], [0], color=colors['Specificity'], lw=2, label='Specificity'),
    plt.Line2D([0], [0], color=colors['AUC'], lw=2, label='AUC')
]
ax.legend(handles=legend_handles, loc='best', bbox_to_anchor=(1, 0.5), fontsize=20, frameon=False)

# Plot customizations
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

plt.show()

In [ ]:
# Updated order for models
models = [
    'NN-AE: resampling 65s', 'LOF: resampling 65s', 'DBSCAN: resampling 65s', 'IF: resampling 65s',
    'NN-AE: resampling 90s', 'LOF: resampling 90s', 'DBSCAN: resampling 90s', 'IF: resampling 90s',
    'NN-AE: without resampling', 'LOF: without resampling', 'DBSCAN: without resampling', 'IF: without resampling'
]

# Metrics updated in the specified order
accuracies = [0.9985, 0.6683, 0.6600, 0.6761, 0.9977, 0.6683, 0.6600, 0.6762, 0.9976, 0.6683, 0.6600, 0.6760]
f1_scores = [0.9977, 0.0078, 0.0047, 0.0030, 0.9980, 0.0078, 0.0047, 0.0033, 0.9963, 0.0078, 0.0047, 0.0021]
recalls = [1.0, 0.0040, 0.0024, 0.0015, 1.0, 0.0040, 0.0024, 0.0016, 1.0, 0.0040, 0.0024, 0.0010]
precisions = [0.9854, 0.1316, 0.0469, 1.0, 0.9861, 0.1316, 0.0469, 1.0, 0.9927, 0.1316, 0.0469, 1.0]
specificities = [0.9978, 0.9871, 0.9757, 1.0, 0.9981, 0.9871, 0.9757, 1.0, 0.9965, 0.9871, 0.9757, 1.0]
auc_scores = [0.9989, 0.4956, 0.4890, 0.5007, 0.9990, 0.4956, 0.4890, 0.5008, 0.9983, 0.4956, 0.4890, 0.5005]

# Confidence intervals for each metric (matching the new order)
ci_acc = [0.0002, 0.0013, 0.0012, 0.0012, 0.0002, 0.0012, 0.0012, 0.0012, 0.0013, 0.0013, 0.0011, 0.0011]
ci_f1 = [0.0002, 0.0006, 0.0004, 0.0003, 0.0003, 0.0005, 0.0004, 0.0004, 0.0006, 0.0006, 0.0004, 0.0003]
ci_recall = [0, 0.0003, 0.0002, 0.0002, 0, 0.0003, 0.0002, 0.0002, 0.0003, 0.0003, 0.0002, 0.0001]
ci_precision = [0.0005, 0.0084, 0.0040, 0, 0.0006, 0.0091, 0.0040, 0, 0.0092, 0.0092, 0.0036, 0]
ci_specificity = [0.0003, 0.0003, 0.0004, 0, 0.0003, 0.0003, 0.0005, 0, 0.0002, 0.0003, 0.0004, 0]
ci_auc = [0.0001, 0.0002, 0.0003, 0.0001, 0.0001, 0.0002, 0.0003, 0.0001, 0.0001, 0.0002, 0.0003, 0.0001]

# Colors for different metrics
colors = {
    'Accuracy': 'red',
    'F1-Score': 'blue',
    'Recall': 'green',
    'Precision': 'purple',
    'Specificity': 'orange',
    'AUC': 'brown'
}

# Create plot
fig, ax = plt.subplots(figsize=(14, 14))

# Plot each model with its metrics and confidence intervals
for i in range(len(models)):
    base_pos = i * 7
    ax.errorbar(accuracies[i], base_pos + 5, xerr=ci_acc[i], fmt='o', color=colors['Accuracy'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(f1_scores[i], base_pos + 4, xerr=ci_f1[i], fmt='o', color=colors['F1-Score'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(recalls[i], base_pos + 3, xerr=ci_recall[i], fmt='o', color=colors['Recall'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(precisions[i], base_pos + 2, xerr=ci_precision[i], fmt='o', color=colors['Precision'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(specificities[i], base_pos + 1, xerr=ci_specificity[i], fmt='o', color=colors['Specificity'], capsize=2, capthick=1, markersize=4)
    ax.errorbar(auc_scores[i], base_pos, xerr=ci_auc[i], fmt='o', color=colors['AUC'], capsize=2, capthick=1, markersize=4)

# Customizing the plot
ax.set_yticks([i * 7 + 2.5 for i in range(len(models))])
ax.set_yticklabels(models, fontsize=20)
ax.set_xticks(np.arange(0, 1.1, 0.1))
ax.set_xlabel('Percentage [x100%]', fontsize=20)
ax.set_ylabel('', fontsize=20)
ax.grid(True, linestyle='--', alpha=0.7, axis='x')
ax.xaxis.set_tick_params(labelsize=15)

# Adding horizontal lines to separate models
for i in range(1, len(models)):
    if i == 4 or i == 8:
        ax.axhline(y=i * 7 - 0.5, color='black', linestyle='-', linewidth=2)  # Bold line
    else:
        ax.axhline(y=i * 7 - 0.5, color='black', linestyle='--', linewidth=1)

# Add a legend outside the plot
legend_handles = [
    plt.Line2D([0], [0], color=colors['Accuracy'], lw=2, label='Accuracy'),
    plt.Line2D([0], [0], color=colors['F1-Score'], lw=2, label='F1-Score'),
    plt.Line2D([0], [0], color=colors['Recall'], lw=2, label='Recall'),
    plt.Line2D([0], [0], color=colors['Precision'], lw=2, label='Precision'),
    plt.Line2D([0], [0], color=colors['Specificity'], lw=2, label='Specificity'),
    plt.Line2D([0], [0], color=colors['AUC'], lw=2, label='AUC')
]
ax.legend(handles=legend_handles, loc='best', bbox_to_anchor=(1, 0.5), fontsize=20, frameon=False)

# Plot customizations
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

plt.show()


Fish detection plots

In [ ]:
# Plots fish detections for a specific date with normal and anomaly detections.
def plot_fish_detections(data, date, legendx=None, xlabelx=None):

    # Convert 'datetime' to datetime type if not already
    data['datetime'] = pd.to_datetime(data['datetime'])

    # Filter data for the specific date
    specific_day_data = data[data['datetime'].dt.date == pd.to_datetime(date).date()]

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 3))

    # Initialize legend handles
    normal_handle = None
    anomaly_handle = None

    # Loop through each detection to plot a line and a circle
    for _, row in specific_day_data.iterrows():
        time = row['datetime']
        if row['Anomaly'] == 0:
            color = 'red'
            label = 'Normal' if normal_handle is None else "_nolegend_"
            normal_handle = True
        else:  # Anomaly detection
            color = 'blue'
            label = 'Anomaly' if anomaly_handle is None else "_nolegend_"
            anomaly_handle = True

        # Plot a line from x-axis to the point
        plt.plot([time, time], [0, 1], color=color, label=label)

        # Plot a circle on top of the line
        plt.scatter([time], [1], color=color, s=30)

    # Formatting the plot
    ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax.xaxis.set_tick_params(labelsize=11)
    plt.title(f'Fish Detections on {date}', fontsize=15)
    if xlabelx is not None:
      plt.xlabel('Time of the day', fontsize=15)

    # Adding the label for the y-axis as 'Detections'
    plt.ylabel('Detections', fontsize=15)

    # Hide y-axis tick values but keep the label
    ax.set_yticks([])

    # Add a legend in the top-right corner
    if legendx is not None:
      plt.legend(loc='best', fontsize=15)

    plt.tight_layout()
    plt.show()


In [ ]:
# Plots for fish detection
plot_fish_detections(data, '2017-01-15',legendx='y', xlabelx=None)
plot_fish_detections(data, '2016-12-22',legendx=None, xlabelx=None)

plot_fish_detections(data, '2016-12-10',legendx=None, xlabelx='y')

In [ ]:
# Ensure 'datetime' column is in datetime format
data['datetime'] = pd.to_datetime(data['datetime'])

# Specify the date
specific_date_2016_12_22 = pd.to_datetime('2016-12-22').date()
specific_date_2017_01_15 = pd.to_datetime('2017-01-15').date()
specific_date_2016_12_10 = pd.to_datetime('2016-12-10').date()

# Filter data for normal points for each day day
normal_data_2016_12_22 = data[
    (data['datetime'].dt.date == specific_date_2016_12_22) & (data['Anomaly'] == 0)
]

normal_data_2017_01_15 = data[
    (data['datetime'].dt.date == specific_date_2017_01_15) & (data['Anomaly'] == 0)
]

normal_data_2016_12_10 = data[
    (data['datetime'].dt.date == specific_date_2016_12_10) & (data['Anomaly'] == 0)
]

In [ ]:
def plot_resampled_detections(augmented_data, date, xlabelx=None):

    # Filter data for the specific date
    specific_day_data = augmented_data[augmented_data['datetime'].dt.date == pd.to_datetime(date).date()]

    # Initialize handles for the legend
    normal_handle = None
    anomaly_handle = None

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 3))
    for _, row in specific_day_data.iterrows():
        time = row['datetime']
        if row['Anomaly'] == 0:  # Normal detection
            color = 'red'
            label = 'Normal' if normal_handle is None else "_nolegend_"
            normal_handle = True
        else:  # Anomaly detection
            color = 'blue'
            label = 'Anomaly' if anomaly_handle is None else "_nolegend_"
            anomaly_handle = True

        # Plot a line from x-axis to the point
        plt.plot([time, time], [0, 1], color=color, label=label)

        # Plot a circle on top of the line
        plt.scatter([time], [1], color=color, s=30)

    # Formatting the plot
    ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax.xaxis.set_tick_params(labelsize=11)
    plt.title(f'Resampled Normal Fish Detections on {date}', fontsize=15)
    if xlabelx is not None:
      plt.xlabel('Time of the day', fontsize=15)

    # Adding the label for the y-axis as 'Detections'
    plt.ylabel('Detections', fontsize=15)

    # Hide y-axis tick values but keep the label
    ax.set_yticks([])

    # Adding the legend at the top-right corner
    #plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
#Resampled Normal Fish Detections on 2017-01-15
augmented_data_Normal_only_withNewAnomaly = augment_data_with_sampling(normal_data_2017_01_15)
plot_resampled_detections(augmented_data_Normal_only_withNewAnomaly, '2017-01-15', xlabelx=None)


#Resampled Normal Fish Detections on 2016-12-22
augmented_data_Normal_only_withNewAnomaly = augment_data_with_sampling(normal_data_2016_12_22)
plot_resampled_detections(augmented_data_Normal_only_withNewAnomaly, '2016-12-22', xlabelx=None)


#Resampled Normal Fish Detections on 2016-12-10
augmented_data_Normal_only_withNewAnomaly = augment_data_with_sampling(normal_data_2016_12_10)
plot_resampled_detections(augmented_data_Normal_only_withNewAnomaly, '2016-12-10', xlabelx='y')

In [ ]:
# Define the longitude and latitude arrays for stations
station_longitude = [20.55331, 20.60169, 20.6057, 20.64561, 20.64984, 20.68885, 20.68968, 20.71504, 20.7181, 20.75242, 20.75805, 20.79069, 20.79869, 20.807021, 20.82168, 20.83526, 20.84281]
station_latitude = [-34.26002, -34.35186, -34.34874, -34.35479, -34.3523, -34.35346, -34.35905, -34.36976, -34.37348, -34.39396, -34.39456, -34.40693, -34.40984, -34.406908, -34.3987, -34.39735, -34.40792]
station_labels = ['BT', 'B14', 'B13', 'B12', 'B11', 'B10', 'B9', 'B8', 'B7', 'B6', 'B5', 'B4', 'B3', 'BC', 'B2', 'Bd', 'B0']
station_colors = ['green'] * len(station_longitude)  # Initialize all stations as green

# Define the stations to change the color
stations_to_change = ['BT', 'B12', 'B11', 'B10', 'B9', 'B8', 'B7', 'B6', 'B4', 'B3', 'BC', 'B2', 'Bd', 'B0']

# Change the color of specified stations to red
for i, label in enumerate(station_labels):
    if label in stations_to_change:
        station_colors[i] = 'red'

# Load the fish movement data
df = data

# Filter the data for fishid "A69-9001-23861a"
fish_data = df.loc[df['fishid'] == "A69-9001-23769"]

# Create a plot for the fish
plt.figure(figsize=(6, 5))  # Adjust the figure size as per your preference
plt.plot(fish_data['lon'], fish_data['lat'])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Fish A69-9001-23769 Movement with Stations')

# Scatter plot for stations with labels in the top right corner
for lon, lat, label, color in zip(station_longitude, station_latitude, station_labels, station_colors):
    plt.scatter(lon, lat, color=color)
    plt.annotate(label, (lon, lat), fontsize=8, ha='left', va='bottom')

# Add legend for green and red dots
plt.scatter([], [], color='green', label='Fish Detected')
plt.scatter([], [], color='red', label='Fish Not Detected')
plt.legend(loc='upper right')

plt.show()

In [ ]:
# Define the longitude and latitude arrays for stations
station_longitude = [20.55331, 20.60169, 20.6057, 20.64561, 20.64984, 20.68885, 20.68968, 20.71504, 20.7181, 20.75242, 20.75805, 20.79069, 20.79869, 20.807021, 20.82168, 20.83526, 20.84281]
station_latitude = [-34.26002, -34.35186, -34.34874, -34.35479, -34.3523, -34.35346, -34.35905, -34.36976, -34.37348, -34.39396, -34.39456, -34.40693, -34.40984, -34.406908, -34.3987, -34.39735, -34.40792]
station_labels = ['BT', 'B14', 'B13', 'B12', 'B11', 'B10', 'B9', 'B8', 'B7', 'B6', 'B5', 'B4', 'B3', 'BC', 'B2', 'Bd', 'B0']
station_colors = ['green'] * len(station_longitude)  # Initialize all stations as green

# Define the stations to change the color
stations_to_change = ['BT', 'B14', 'B13', 'B12', 'B11', 'B10', 'B9', 'B8', 'B7', 'B6', 'B5', 'B4', 'BC', 'B2', 'Bd', 'B0']

# Change the color of specified stations to red
for i, label in enumerate(station_labels):
    if label in stations_to_change:
        station_colors[i] = 'red'

# Load the fish movement data
df = data

# Filter the data for fishid "A69-9001-23702"
fish_data = df.loc[df['fishid'] == "A69-9001-23702"]

# Create a plot for the fish
plt.figure(figsize=(6, 5))  # Adjust the figure size as per your preference
plt.plot(fish_data['lon'], fish_data['lat'])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Fish A69-9001-23702 Movement with Stations')

# Scatter plot for stations with labels in the top right corner
for lon, lat, label, color in zip(station_longitude, station_latitude, station_labels, station_colors):
    plt.scatter(lon, lat, color=color)
    plt.annotate(label, (lon, lat), fontsize=8, ha='left', va='bottom')

# Add legend for green and red dots
plt.scatter([], [], color='green', label='Fish Detected')
plt.scatter([], [], color='red', label='Fish Not Detected')
plt.legend(loc='upper right')

plt.show()